### Imports, Globals, and Configuration

In [2]:
# Imports and globals
import pandas as pd
import numpy as np
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

CONFIG_PATH="/Users/tomkusak/Desktop/School/data_science/flight_delays/config/config.json"
YEARS=["2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]

### Define Functions

In [3]:
# Define functions
# Load configuration
with open(CONFIG_PATH) as fp:
    config = json.load(fp)

def get_df(year):
    '''
    Returns a dataframe for a specific year.
    '''
    file_path = f'{config["data_path"]}/{year}.csv'
    return pd.read_csv(file_path)

### Load DataFrame Dictionary (heavy compute)

In [4]:
# Load all data into dictionary of dataframes
df_dict = {}

# Takes a fat minute or FOUR (3:53.8)
for year in YEARS:
    df_dict[year] = get_df(year)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

### Perform Analysis

In [5]:
# going to use test file for now
df = pd.read_csv(f'{config["data_path"]}/test.csv')

In [ ]:
scaler = StandardScaler()
# Make sure our DEP_TIME and ORIGIN are numeric
df['DEP_TIME'] = pd.to_numeric(df['DEP_TIME'], errors='coerce')
df['ARR_TIME'] = pd.to_numeric(df['ARR_TIME'], errors='coerce')

# Handle missing values in the 'DEP_TIME' and 'ORIGIN' columns
df['DEP_TIME'].fillna(df['DEP_TIME'].mean(), inplace=True)
df['ARR_TIME'].fillna(df['ARR_TIME'].mean(), inplace=True)

# Collect x and y values
X = df[['DEP_TIME', 'ARR_TIME']].values
y = df['CANCELLED'].values


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Train the k-NN model
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

def plot_decision_boundaries(X, y, model, title='Decision Boundaries'):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(('red', 'green')))
    plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k', cmap=ListedColormap(('red', 'green')))
    plt.title(title)
    plt.xlabel('DEP_TIME')
    plt.ylabel('ARR_TIME')
    plt.show()

# Plot decision boundaries on the testing set
plot_decision_boundaries(X_test, y_test, knn, title='Decision Boundaries (Test Set)')

# Make predictions and evaluate the model
y_pred = knn.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Prediction accuracy = {100 * accuracy}%")

: 